In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
from scipy.stats import gaussian_kde as gkde
from IPython.display import Markdown, clear_output
import ipywidgets as widgets
%matplotlib inline
%matplotlib widget
# plt.close('all')

# plt.style.use("default")
#OR
# plt.style.use("dark_background")
# plt.rcParams['figure.facecolor']=.07*np.ones(3)
# plt.rcParams['figure.edgecolor']=.07*np.ones(3)
# plt.rcParams['axes.facecolor']=.07*np.ones(3)  # set theme
# plt.rcParams['grid.linewidth']=.2


In [10]:
df = pd.read_csv("/Datasets/IntelData.csv")
allSKUs = df.SKU.unique()
dCenters = df['Distribution Center'].unique()
display(Markdown(f"The dataset consists of **{len(df):,}** rows, with **{len(allSKUs)}** individual SKUs over **{len(dCenters)}** distribution centers") )

The dataset consists of **26,114** rows, with **86** individual SKUs over **5** distribution centers

In [33]:
class dataSummary():
    """
    This class is to display a summary of the Intel Dataset.
    """
    # The constructor is used to instantiate the methods.
    def __init__(self):
        self.set_widgets()
        self.set_figure()
        self.update_data_plot(1)
    
    # The set_widgets method is used to initialize the widgets required.
    # Documentation : https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
    def set_widgets(self):
        
        #All widgets are suffixed with a '_w'
        
        self.sku_w = widgets.Dropdown(options=allSKUs,description="SKU")
        self.dCenter_w = widgets.SelectMultiple(options=dCenters,value=[dCenters[0],],description="Distribution Center")
        self.showData_w = widgets.Checkbox(description="Show Data",value=True)
        self.yScale_w = widgets.RadioButtons(options=["log","linear"],description="$y$-scale")
        self.outputFig_w = widgets.Output()
        self.display_w = widgets.Output()
        
        #Set up the display of widgets
        
        display( 
            widgets.VBox(
                 [ 
                    widgets.HBox([self.dCenter_w,self.sku_w,self.yScale_w,self.showData_w]),
                    widgets.HBox([self.outputFig_w,self.display_w])
                 ]
                        )
               )
        
        #Let's use the observe method to handle changes to the widgets.
        #Documentation : https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Registering-callbacks-to-trait-changes-in-the-kernel
        
        for eachWidget_w in [self.sku_w,self.dCenter_w,self.showData_w,self.yScale_w]:
            eachWidget_w.observe(self.update_data_plot,names="value")
    
    # The set_figure method is used to generate the figure, axes, and labels.    
    def set_figure(self):
        with self.outputFig_w:
            with plt.ioff():
                num = "SKU orders and forecasts"
                if plt.fignum_exists(num):plt.close(num)
                self.fig,self.ax = plt.subplots(
                    num=num, ncols=3,sharey=True,figsize=(9,3.5),
                    gridspec_kw=
                                dict(width_ratios=[4,1,1],
                                     wspace=0.05,
                                     top = 0.99, 
                                     right = 0.99, 
                                     bottom = 0.14)
                )

                self.forecast_plot, = self.ax[0].plot([],[],label="Forecasts",lw=0,marker=".",alpha=0.4)
                self.orders_plot, = self.ax[0].plot([],[],label="Orders",lw=0,marker=".",alpha=0.4)
                self.ax[0].legend()
                self.ax[0].set(xlabel="Time $t$ [Weeks]",
                               ylabel="Forecasts & Orders [Count]",
                               yscale=self.yScale_w.value)

                self.fig.canvas.header_visible=False #Hides the figure header Eg: "Figure 14"
                self.fig.canvas.toolbar_visible=False
                self.fig.canvas.show()  
                
    
    # The update_data_plot method is used to update the plots for each widget action taken.
    # @event - ipywidget event
    def update_data_plot(self,event):
        
        # We first obtain the corresponding data for the selected DC and SKU.
        df_plotMe = df[(df.SKU==self.sku_w.value) & (df["Distribution Center"].isin(self.dCenter_w.value))]
        
        # Now, we can update the plots with the corresponding data selected above.
        self.forecast_plot.set_data(df_plotMe.Week,df_plotMe["Forecasted Demand"])
        self.orders_plot.set_data(df_plotMe.Week,df_plotMe["Customer Orders"])
        self.ax[0].set_xlim(0,df_plotMe.Week.values.max()) #X-axis limits
        
        all_ys = np.r_[df_plotMe["Forecasted Demand"].values,df_plotMe["Customer Orders"].values]
        if self.yScale_w.value == "lin":
            ylim = [0,all_ys.max()]
        else:
            ylim = [all_ys.min()/2,all_ys.max()*2]
        self.ax[0].set(yscale=self.yScale_w.value)
        self.ax[0].set_ylim(ylim) #Y-axis limits
        
        # Next, we can update the distribution histogram plots.
        self.show_distribution(df_plotMe["Forecasted Demand"].values,self.ax[1],color="C0")
        self.show_distribution(df_plotMe["Customer Orders"].values,self.ax[2],color="C1")
        self.fig.canvas.draw()
        
        # Display the data if selected.
        with self.display_w:
            clear_output()
            if self.showData_w.value:
                display(df_plotMe.head())
                
                
                
    # The show_distribution method is used to display the histogram of Forecasts and Orders.
    def show_distribution(self,x,ax_,color):
        try:
                if color=="C0":
                    _ = [b.remove() for b in self.bars1]
                else:
                    _ = [b.remove() for b in self.bars2]
        except:
                pass
            
        x = x[x>0]
        if len(x)>2:
            m,M = x.min(),x.max()
            p_orders = gkde(x)
            if self.yScale_w.value == "log":
                bins = np.logspace(np.log10(m),np.log10(M),20)
                if color == "C0":
                    c,_,self.bars1 = ax_.hist(x,bins,orientation="horizontal",label="Forecast",alpha=.4,density=False,color=color)
                else:
                    c,_,self.bars2=ax_.hist(x,bins,orientation="horizontal",label="Forecast",alpha=.4,density=False,color=color)
            elif self.yScale_w.value == "linear":
                bins = np.linspace(m,M,20)
                x_ = np.linspace(m,M,200)
                if color == "C0":
                        c,_,self.bars1=ax_.hist(x,bins,orientation="horizontal",label="Forecast",alpha=.4,density=True,color=color)
                else:
                        c,_,self.bars2=ax_.hist(x,bins,orientation="horizontal",label="Forecast",alpha=.4,density=True,color=color)
                        
            ax_.set(xlim=[0,1.05*c.max()])
                
            

                    
                
intelViewer = dataSummary()        